In [1]:
%%shell
pip install bitsandbytes
pip install accelerate
pip install trl peft
pip install datasets
pip install rouge-score
pip install evaluate
pip install huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [2]:
from google.colab import drive, userdata
from huggingface_hub import login

import os
import json
#mounting google drive
drive.mount('/content/drive')

########################################

#changing the working directory
os.chdir("/content/drive/MyDrive/NLP_Data")

!pwd

huggingface_token = userdata.get('Hugging_Face_Hub_API_TOKEN')

#logging into huggingface
login(huggingface_token)

Mounted at /content/drive
/content/drive/MyDrive/NLP_Data
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging

import evaluate

from sklearn.model_selection import train_test_split


In [4]:
df = pd.read_csv('/content/drive/MyDrive/NLP_Data/train_valid_data.csv')
df.head()

,channel_name,video_title,video_description,comment_text,like_count,reply_count,Sentiment,Explanation
0,BBC,Can Cuttlefish camouflage in a living room? | ...,The final episode of Richard Hammond’s Miracle...,The big white square on his back was impressiv...,482.0,3.0,Positive,The comment expresses admiration for the cuttl...
1,BBC,"""I Cut Off My Own Arm To Save My Life"" | Pleas...",Pleasure is vital for our survival - without i...,"The truck may be strong, but family is stronger",0.0,0.0,Positive,The comment highlights the importance and stre...
2,The Late Show with Stephen Colbert,Elon Musk Might Be A Super Villain,The Late Show with Stephen Colbert is broadcas...,His mentality is genius. The way people look a...,820.0,22.0,Positive,This comment expresses admiration for Elon Mus...
3,hbomberguy,"Bloodborne Is Genius, And Here's Why","Channel: hbomberguy, Title: Bloodborne Is Geni...","""To my recollection, Ludwig's Holy Blade is th...",82.0,2.0,Neutral,"This comment is purely informative, providing ..."
4,Mentour Pilot,What REALLY Caused the Tenerife Airport Disast...,Mentour Pilot: What REALLY Caused the Tenerife...,Insurance fraud?,0.0,0.0,Negative/Speculative,The comment suggests a possible motive of insu...


In [5]:
print(
    f"""
        Channel Name: {df['channel_name'][0]}
        Video Title: {df['video_title'][0]}
        Description: {df['video_description'][0]}
        Comment Text: {df['comment_text'][0]}
        \n
        Sentiment: {df['Sentiment'][0]}
        Explanaition: {df['Explanation'][0]}


    """
)


        Channel Name: BBC
        Video Title: Can Cuttlefish camouflage in a living room? | Richard Hammond's Miracles of Nature - BBC
        Description: The final episode of Richard Hammond’s Miracles Of Nature. Richard is once again investigating the extraordinary super-powers of the animal kingdom. Cuttlefish survive by being able to blend into their surroundings through camouflage. Richard Hammond puts this to the test and experiments if the fish are able to camouflage in a tank set up like a living room.
        Comment Text: The big white square on his back was impressive af even tho it wasn't fooling our human perception.
        

        Sentiment: Positive
        Explanaition: The comment expresses admiration for the cuttlefish's camouflage abilities, despite it not being completely convincing to humans.

    
    


Load and Preprocess the Data

In [6]:
# # Load datasets
# test_df = pd.read_csv('/content/drive/MyDrive/NLP_Data/test_data.csv')
# train_valid_data = pd.read_csv('/content/drive/MyDrive/NLP_Data/train_valid_data.csv')

# # Split the dataset into training and validation sets (80-20 split)
# train_df, val_df = train_test_split(train_valid_data, test_size=0.2, random_state=42)

# # Format the data for training
# formatted_train_data = [
#     {
#         "input": f"Channel: {row['channel_name']}, Title: {row['video_title']}, Description: {row['video_description']}, Comment Text: {row['comment_text']}",
#         "output": f"Sentiment: {row['Sentiment']}, Explanation: {row['Explanation']}"
#     }
#     for _, row in train_df.iterrows()
# ]

# # Format the data for validation
# formatted_val_data = [
#     {
#         "input": f"Channel: {row['channel_name']}, Title: {row['video_title']}, Description: {row['video_description']}, Comment Text: {row['comment_text']}",
#         "output": f"Sentiment: {row['Sentiment']}, Explanation: {row['Explanation']}"
#     }
#     for _, row in val_df.iterrows()
# ]

# # Format the data for test
# formatted_test_data = [
#     {
#         "input": f"Channel: {row['channel_name']}, Title: {row['video_title']}, Description: {row['video_description']}, Comment Text: {row['comment_text']}",
#         "output": "Sentiment: , Explanation: "
#     }
#     for _, row in test_df.iterrows()
# ]

# # Convert formatted data to Dataset objects
# train_dataset = Dataset.from_list(formatted_train_data)
# val_dataset = Dataset.from_list(formatted_val_data)
# test_dataset = Dataset.from_list(formatted_test_data)


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
# # Load Data
# test_df = pd.read_csv('/content/drive/MyDrive/NLP_Data/test_data.csv')
# train_valid_data = pd.read_csv('/content/drive/MyDrive/NLP_Data/train_valid_data.csv')

# # Split the dataset into training and validation sets (80-20 split)
# train_df, val_df = train_test_split(train_valid_data, test_size=0.2, random_state=42)

# # Initialize tokenizer and model
# model_name = "facebook/bart-large-cnn"  # BART model name
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)

# # Data Preparation
# def format_data(df, for_test=False):
#     return [
#         {
#             "input": f"Channel: {row['channel_name']}, Title: {row['video_title']}, Description: {row['video_description']}, Comment Text: {row['comment_text']}",
#             "output": f"Sentiment: {row['Sentiment']}, Explanation: {row['Explanation']}" if not for_test else "Sentiment: , Explanation: "
#         }
#         for _, row in df.iterrows()
#     ]

# # Format the data
# formatted_train_data = format_data(train_df)
# formatted_val_data = format_data(val_df)
# formatted_test_data = format_data(test_df, for_test=True)

# # Convert to Dataset objects
# train_dataset = Dataset.from_list(formatted_train_data)
# val_dataset = Dataset.from_list(formatted_val_data)
# test_dataset = Dataset.from_list(formatted_test_data)

# # Tokenization
# def tokenize_data(example):
#     model_inputs = tokenizer(
#         example["input"],
#         max_length=512,
#         padding="max_length",
#         truncation=True
#     )
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(
#             example["output"],
#             max_length=128,
#             padding="max_length",
#             truncation=True
#         )
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # Tokenize datasets
# tokenized_train_dataset = train_dataset.map(tokenize_data, batched=True)
# tokenized_val_dataset = val_dataset.map(tokenize_data, batched=True)

# # Configure LoRA
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
# )
# lora_model = get_peft_model(model, lora_config)

# # Define Training Arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=6,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     warmup_steps=500,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_steps=1000,
#     evaluation_strategy="steps",
#     eval_steps=500,
# )

# # Define Evaluation Metric and Compute Function
# rouge_metric = evaluate.load('rouge')  # Load the metric with evaluate

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds = ["\n".join(pred.strip().split(". ")) for pred in decoded_preds]
#     decoded_labels = ["\n".join(label.strip().split(". ")) for label in decoded_labels]

#     result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     return result

# # Initialize Trainer
# trainer = Trainer(
#     model=lora_model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_val_dataset,
#     compute_metrics=compute_metrics
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# eval_results = trainer.evaluate()
# print(f"Evaluation Results: {eval_results}")

# # Save the Fine-Tuned Model
# lora_model.save_pretrained("./SA-bart-fine-tuned-lora-model")
# tokenizer.save_pretrained("./SA-bart-fine-tuned-lora-model")

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
500,1.628000,No log


Evaluation Results: {'eval_runtime': 11.1202, 'eval_samples_per_second': 8.993, 'eval_steps_per_second': 2.248, 'epoch': 6.0}


('./SA-bart-fine-tuned-lora-model/tokenizer_config.json',
 './SA-bart-fine-tuned-lora-model/special_tokens_map.json',
 './SA-bart-fine-tuned-lora-model/vocab.json',
 './SA-bart-fine-tuned-lora-model/merges.txt',
 './SA-bart-fine-tuned-lora-model/added_tokens.json')

In [15]:
# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained("./SA-bart-fine-tuned-lora-model")
tokenizer = BartTokenizer.from_pretrained("./SA-bart-fine-tuned-lora-model")

# Load the test data
test_df = pd.read_csv('/content/drive/MyDrive/NLP_Data/test_data.csv')

# Prepare the test data for inference
def format_test_data(df):
    return [
        {
            "input": f"Channel: {row['channel_name']}, Title: {row['video_title']}, Description: {row['video_description']}, Comment Text: {row['comment_text']}",
            "output": ""  # For test data, the output is not needed
        }
        for _, row in df.iterrows()
    ]

# Format and convert test data to Dataset
formatted_test_data = format_test_data(test_df)
test_dataset = Dataset.from_list(formatted_test_data)

# Tokenize the test data
def tokenize_data(example):
    model_inputs = tokenizer(
        example["input"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    return model_inputs

# Tokenize the test dataset
tokenized_test_dataset = test_dataset.map(tokenize_data, batched=True)

# Inference
def infer(model, tokenizer, dataset):
    model.eval()
    predictions = []
    for example in dataset:
        inputs = tokenizer(example['input'], return_tensors='pt', truncation=True, padding=True, max_length=512)
        outputs = model.generate(**inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(prediction)
    return predictions

# Get predictions
predictions = infer(model, tokenizer, tokenized_test_dataset)




Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
predictions[8]

"Sentiment: Negative, Explanation: The comment expresses negative feelings about Tom Hanks, suggesting a negative reaction to the film, suggesting that the actor's dislike of Hanks is the biggest crime of all. The comment has a negative tone, suggesting negative feelings toward Hanks and the film."

In [8]:
# Add predictions to the test DataFrame
test_df['Predicted_Sentiment'] = predictions

# Save or display results
test_df.to_csv('/content/drive/MyDrive/NLP_Data/test_predictions.csv', index=False)
print(test_df[['comment_text', 'Predicted_Sentiment']])

In [5]:
# prompt: read the video_data
import pandas as pd
video_data = pd.read_csv('/content/drive/MyDrive/NLP_Data/video_data_df.csv')
video_data.head()


,channel_name,video_title,video_description,comment_text,like_count,reply_count
0,BBC,Can Cuttlefish camouflage in a living room? | ...,The final episode of Richard Hammond’s Miracle...,The big white square on his back was impressiv...,482.0,3.0
1,BBC,"""I Cut Off My Own Arm To Save My Life"" | Pleas...",Pleasure is vital for our survival - without i...,"The truck may be strong, but family is stronger",0.0,0.0
2,The Late Show with Stephen Colbert,Elon Musk Might Be A Super Villain,The Late Show with Stephen Colbert is broadcas...,His mentality is genius. The way people look a...,820.0,22.0
3,hbomberguy,"Bloodborne Is Genius, And Here's Why","Channel: hbomberguy, Title: Bloodborne Is Geni...","""To my recollection, Ludwig's Holy Blade is th...",82.0,2.0
4,Mentour Pilot,What REALLY Caused the Tenerife Airport Disast...,Mentour Pilot: What REALLY Caused the Tenerife...,Insurance fraud?,0.0,0.0


In [6]:
# prompt: print columns of video_data

print(video_data.columns)


Index(['channel_name', 'video_title', 'video_description', 'comment_text',
       'like_count', 'reply_count'],
      dtype='object')
